In [ ]:
import random
from PIL import Image
import numpy as np

import torch
from  torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0")# if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [ ]:
total = 3683
name = []
labels = {}
for i in range(total):
    item = "KR_k-" + str(i/1000) + ".png"
    name.append(item)
    labelsi = {name[i] : name[i][5:-4]}
    labels.update(labelsi)

random.shuffle(name)
partition = {'train': name[:int(len(name)*0.8)], 'validation': name[int(len(name)*0.8):int(len(name)*0.9)], 'test': name[int(len(name)*0.9):]}

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)


  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        transform = transforms.Compose([transforms.PILToTensor()])
        X = transform(Image.open('./Dados_2/' + ID).resize((100, 100)))/255.
        y = self.labels[ID]

        return X.to(device), float(y)


https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

https://www.youtube.com/watch?v=NVxCKdp0NhQ

In [ ]:
# Parameters
params = {'batch_size': 32,
          'shuffle': True}
max_epochs = 100

# Generators
training_set = Dataset(partition['train'], labels)
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = Dataset(partition['validation'], labels)
validation_generator = torch.utils.data.DataLoader(validation_set, **params)

In [ ]:
class KR(nn.Module):
    def __init__(self):
        super(KR, self).__init__()
        self.conv = nn.Sequential(

            nn.Conv2d(4, 8, 3, padding=1), 
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2, stride=2), #50

            nn.Conv2d(8, 16, 3, padding=1), 
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, stride=2), #25

            nn.Conv2d(16, 32, 3, padding=1), 
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, stride=2)) #12
        
        self.linear = nn.Sequential(

            nn.Linear(32*12*12,128),
            nn.LeakyReLU(0.1),

            nn.Linear(128,1),
            nn.LeakyReLU(0.1),

            nn.Linear(1,1),
            nn.LeakyReLU(0.1))
    
    def forward(self, x):
        h = self.conv(x)
        #r = torch.reshape(h, (-1,))
        h = h.view(-1, 32*12*12)
        h = self.linear(h)
        return h

In [ ]:
kr = KR()
kr.to(device)

In [ ]:
optimizer = torch.optim.Adam(kr.parameters(), lr=2.5E-3)
loss = torch.nn.L1Loss()
max_epochs = 300

In [ ]:
# Loop over epochs
idx = 0
batch_size = 16
losses = []
loss_per_epoch = []
val_loss_per_epoch = []
val_losses = []

for epoch in range(max_epochs):
    # Training
    for local_batch, local_labels in training_generator:
        idx += 1
        # take a batch
        local_batch.to(device)
        local_labels = torch.Tensor(list(local_labels)).to(device)
        optimizer.zero_grad()
        # forward pass
        input = kr(local_batch).view(-1)
        output = torch.nn.functional.mse_loss(input, local_labels)
        # backward pass
        output.backward()
        # update weights
        optimizer.step()

        losses.append(float(output))

        #if idx%100 == 0:
    loss_per_epoch.append(float(output))

    # Validation
    with torch.set_grad_enabled(False):
        for vlocal_batch, vlocal_labels in validation_generator:
            vlocal_batch.to(device)
            vlocal_labels = torch.Tensor(list(vlocal_labels)).to(device)

            vinput = kr(vlocal_batch).view(-1)

            voutput = torch.nn.functional.mse_loss(vinput, vlocal_labels)


            val_losses.append(float(voutput))
    
    val_loss_per_epoch.append(float(voutput))
    
    print("\tEpoch", epoch +1, "\tAverage Loss: ", output.tolist()/(batch_size), "\tValidation Loss: ", voutput.tolist()/(batch_size))

print('Finished Training')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(np.arange(len(loss_per_epoch)), loss_per_epoch, color='tab:blue', label = 'Train')
ax.plot(np.arange(len(val_loss_per_epoch)), val_loss_per_epoch, color='tab:orange', label = "Val")
plt.show()

In [ ]:
test_set = Dataset(partition['validation'], labels)
test_generator = torch.utils.data.DataLoader(test_set, batch_size = 1, shuffle= True)

for i in range(10):
    tlocal_batch, tlocal_labels = next(iter(test_generator))

    pred = kr(tlocal_batch)
    print(pred) #prediction
    print(tlocal_labels) #true
    print('------------')



In [ ]:
test_set = Dataset(partition['test'], labels)
a = 35
test_generator = torch.utils.data.DataLoader(test_set[a], batch_size = 1)

tlocal_batch, tlocal_labels = test_generator

pred = kr(tlocal_batch)
print(pred)
print(tlocal_labels)
print('------------')
